In [1]:
import pickle
import os
import numpy as np
import pandas as pd

from keras.layers import Dense, Dropout, Embedding
from keras.models import Input, Model
from keras.preprocessing import sequence
from tensorflow.python.framework import ops
from tcn import TCN
from tcn import compiled_tcn
from sklearn.model_selection import train_test_split

ops.reset_default_graph()

Using TensorFlow backend.


In [2]:
data = pickle.load( open( "dfsm.pkl", "rb" ) )

In [62]:
cols=list(data[1].iloc[:,:34].columns)+list(data[1].iloc[:,-6:-1])

In [63]:
cols

['HR',
 'O2Sat',
 'Temp',
 'SBP',
 'MAP',
 'DBP',
 'Resp',
 'EtCO2',
 'BaseExcess',
 'HCO3',
 'FiO2',
 'pH',
 'PaCO2',
 'SaO2',
 'AST',
 'BUN',
 'Alkalinephos',
 'Calcium',
 'Chloride',
 'Creatinine',
 'Bilirubin_direct',
 'Glucose',
 'Lactate',
 'Magnesium',
 'Phosphate',
 'Potassium',
 'Bilirubin_total',
 'TroponinI',
 'Hct',
 'Hgb',
 'PTT',
 'WBC',
 'Fibrinogen',
 'Platelets',
 'Age',
 'Gender',
 'Unit1',
 'Unit2',
 'HospAdmTime']

In [66]:
data[1][cols]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime
ICULOS,,,,,,,,,,,,,,,,,,,,,
1,84.581392,97.193766,36.976678,123.750299,82.400065,63.827028,18.726471,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
2,61.000000,99.000000,36.440000,124.000000,65.000000,43.000000,17.500000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
3,64.000000,98.000000,36.440000,125.000000,64.000000,41.000000,27.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
4,56.000000,100.000000,36.440000,123.000000,65.000000,41.000000,9.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
5,66.000000,99.000000,36.440000,120.000000,67.000000,43.000000,23.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
6,94.000000,100.000000,36.220000,194.000000,116.000000,66.000000,14.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
7,58.000000,99.000000,36.220000,133.000000,68.000000,43.000000,13.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
8,57.000000,100.000000,36.220000,118.000000,62.000000,37.000000,18.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6
9,62.000000,100.000000,36.220000,126.000000,66.000000,37.000000,12.000000,32.912749,-0.678516,24.077791,...,10.428949,41.160659,11.442498,285.236011,196.005664,75.91,0,0,1,-98.6


In [68]:
x=list()
y=list()
for i in range(len(data)):
    x.append(np.asarray(data[i][cols]))
    y.append(np.asarray(data[i].iloc[:,-1:]))

In [ ]:
len(x)

In [72]:
np.expand_dims([np.insert(np.zeros(shape=(5, 1)), 0, 1)], axis=-1)

array([[[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [74]:
np.expand_dims(y_train[0], axis=-1).shape

(336, 1, 1)

In [71]:
np.array([np.zeros(shape=(5, 1))])

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [ ]:
def get_x_y(max_time_steps):
    for k in range(len(x_train)):
        time_steps = len(x_train[k])
        if k % 2 == 0:
            x_train = np.expand_dims([np.insert(np.zeros(shape=(time_steps, 1)), 0, 1)], axis=-1)
            y_train = [1]
        else:
            x_train = np.array([np.zeros(shape=(time_steps, 1))])
            y_train = [0]
        print('\nInput: sequence of length {}\n'.format(time_steps))
        yield x_train, np.expand_dims(y_train, axis=-1)

In [ ]:
def get_x_y(max_time_steps):
    for k in range(int(1e9)):
        time_steps = np.random.choice(range(1, max_time_steps), size=1)[0]
        if k % 2 == 0:
            x_train = np.expand_dims([np.insert(np.zeros(shape=(time_steps, 1)), 0, 1)], axis=-1)
            y_train = [1]
        else:
            x_train = np.array([np.zeros(shape=(time_steps, 1))])
            y_train = [0]
        print('\nInput: sequence of length {}\n'.format(time_steps))
        yield x_train, np.expand_dims(y_train, axis=-1)


In [8]:
x = sequence.pad_sequences([*x], maxlen=336)

In [9]:
y = sequence.pad_sequences([*y], maxlen=336)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [5]:
maxlen=336
max_features=73

In [11]:
model = compiled_tcn(num_feat=73,num_classes=2,nb_filters=15,dilations=[1,2,4,8,16],nb_stacks=2,max_len=None,padding='causal',return_sequences=True,kernel_size=5)

x.shape= (?, ?, 15)
model.x = (?, ?, 73)
model.y = (?, ?, 2)


In [27]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Train on 28235 samples, validate on 12101 samples
Epoch 1/5
28235/28235 [==============================] - 28s 993us/step - loss: 0.0325 - accuracy: 0.9980 - val_loss: 0.0348 - val_accuracy: 0.9978
Epoch 2/5
28235/28235 [==============================] - 27s 967us/step - loss: 0.0325 - accuracy: 0.9980 - val_loss: 0.0348 - val_accuracy: 0.9978
Epoch 3/5
28235/28235 [==============================] - 27s 965us/step - loss: 0.0325 - accuracy: 0.9980 - val_loss: 0.0348 - val_accuracy: 0.9978
Epoch 4/5
28235/28235 [==============================] - 28s 980us/step - loss: 0.0325 - accuracy: 0.9980 - val_loss: 0.0348 - val_accuracy: 0.9978
Epoch 5/5
28235/28235 [==============================] - 29s 1ms/step - loss: 0.0325 - accuracy: 0.9980 - val_loss: 0.0348 - val_accuracy: 0.9978


In [13]:
seps=list()
for i in range(len(y_test)):
    if y_test[i][-1]==1:
        seps.append(i)

In [15]:
preds=list()
for i in range(len(seps)):
    if model.predict(x_test[seps[i]:seps[i]+1,:,:])[0,-1,][0] <0.5:
        preds.append(i)

In [28]:
model.predict(x_test[64:65,:,:])

array([[[9.9999952e-01, 4.3851153e-07],
        [1.0000000e+00, 5.1600640e-08],
        [1.0000000e+00, 9.0825951e-11],
        [1.0000000e+00, 4.8385335e-13],
        [1.0000000e+00, 1.1355056e-17],
        [1.0000000e+00, 3.7897607e-21],
        [1.0000000e+00, 1.3011284e-25],
        [1.0000000e+00, 4.8539116e-29],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00],


In [25]:
seps[2]

64

In [21]:
y_test[8]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [20]:
seps[0],y_test[8])

SyntaxError: invalid syntax (<ipython-input-20-bff6c088584e>, line 1)

In [ ]:
seps=list()
for i in range(len(y_test)):
    if y_test[i][-1]==1:
        seps.append(i)
        

In [ ]:
len(seps)/len(y_test)

In [ ]:
y_test[23]